In [ ]:
# %matplotlib notebook
import pandas as pd
import geopandas as gpd
import requests as r
import os
import typing
import json

# manipulating spatial data
from io import StringIO
import zipfile
import matplotlib.pyplot as plt
import folium
import folium.plugins

In [ ]:
# helper stuff
STATE = 'state'
COUNTY = 'county'
TRACT = 'tract'
BLOCK_GROUP = 'block group'
LEVELS_OF_DETAIL = [STATE, COUNTY, TRACT]

PREFIX_COLUMNS = ['geo_id', 'state', 'county', 'tract', 'block group']

## Census Variables ##

For our purposes, we're interested in the census variable group B08302 - which is when people leave to go to work.

In [ ]:
API_KEY = os.environ.get("CENSUS_API_KEY")
# example url: 
# https://api.census.gov/data/2017/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:01&key=
URL_STUB = "https://api.census.gov/data/2017/acs/acs5?get={variable}&for={level_of_detail}:*&in=state:{fips_state}&in=county:{county}&key={api_key}"


In [ ]:
def fetch_census_variable(variable: str, 
                          fips_state: str, 
                          fips_county: str,
                          level_of_detail: str, 
                          api_key: str, 
                          url_stub: str = URL_STUB) -> typing.Dict:
    """
    Using a census variable, pull the level of detail requested.
    """
    # pull data from REST API
    api_url = url_stub.format(
        variable=variable, 
        fips_state=fips_state, 
        county=county, 
        api_key=API_KEY, 
        level_of_detail=level_of_detail
    )
    api_resp = r.get(api_url)
    
    json_data = api_resp.json()
    keys = json_data[0]
    keys = list(map(lambda x: x.lower(), keys))

    data_set = [dict(zip(keys, elem)) for elem in json_data[1:]]
    # create dataframe
    return pd.DataFrame(data_set)

In [ ]:
variable = "group(B08302)" # Time leaving home to go to work
omaha_cbsa = [
    ('19','155'),
    ('31', '025'),
    ('31', '055'),
    ('31', '153'),
    ('31', '177'),
]

dfs = []

for fips_state, county in omaha_cbsa:
    print('fetching {}{}'.format(fips_state, county))
    # Build a basic data frame
    df = fetch_census_variable(
        variable=variable,
        fips_state=fips_state, 
        fips_county=county, 
        level_of_detail=BLOCK_GROUP, 
        api_key=API_KEY
    )

    dfs.append(df)
    print('found {} data points for {}{}'.format(len(df), fips_state, county))


df = pd.concat(dfs)

df.head(5)

In [ ]:
filter_columns = lambda x: x.startswith('b') and x.endswith('e')

columns = list(set(df.columns).difference(set(PREFIX_COLUMNS)))

retain_columns = sorted(list(filter(filter_columns, columns)))

df2 = df[PREFIX_COLUMNS + retain_columns].copy(deep=True)

for col in filter(filter_columns, df2.columns):
    df2[col] = df2[col].astype('int')
 
df2['geo_id'] = df2['state'] + df2['county'] + df2['tract'] + df2['block group']

In [ ]:
df2.head(10)

# Geospatial Data

The Census Bureau publishes shape files on the state level, which are useful when we're lookind at very detailed geographies such as tracts or block groups.

Download the block groups here:'ftp://ftp2.census.gov/geo/tiger/TIGER2018/BG/tl_2018_31_bg.zip'

In [ ]:
# shape file
ne_shape_file_path = '../ref_data/tl_2018_31_bg/tl_2018_31_bg.shp'
ia_shape_file_path = '../ref_data/tl_2018_19_bg/tl_2018_19_bg.shp'

ne_gdf = gpd.read_file(ne_shape_file_path)
ia_gdf = gpd.read_file(ia_shape_file_path)

ne_gdf = ne_gdf[ne_gdf['COUNTYFP'].isin(('055', '025', '153', '177'))]
ia_gdf = ia_gdf[ia_gdf['COUNTYFP'].isin(('155',))]

cbsa_gdf = pd.concat([ne_gdf, ia_gdf])

cbsa_gdf.head(10)

In [ ]:
f, ax = plt.subplots(1, figsize=(16, 16))
ax = cbsa_gdf.plot(ax=ax)
plt.show()

In [ ]:
cbsa_gdf = cbsa_gdf.merge(df2, right_on='geo_id', left_on='GEOID')

cbsa_gdf.head(4)

In [ ]:
mapping_columns = [
    'geo_id',
    'state',
    'county',
    'tract',
    'block group',
    'geometry',
    'b08302_001e', 
    'b08302_002e', 
    'b08302_003e', 
    'b08302_004e',
    'b08302_005e', 
    'b08302_006e', 
    'b08302_007e',
    'b08302_008e',
    'b08302_009e',
    'b08302_010e',
    'b08302_011e',
    'b08302_012e',
    'b08302_013e',
    'b08302_014e',
    'b08302_015e'
]


variable = 'b08302_005e'


for_map_gdf = cbsa_gdf[mapping_columns][cbsa_gdf['county'].isin(['055'])]
max_amount = float(for_map_gdf[variable].max())
variable_total = float(for_map_gdf[variable].sum())

for_map_gdf['percentage'] = for_map_gdf[variable] / variable_total
for_map_gdf = for_map_gdf[['geo_id', 'geometry', variable, 'percentage']]

map_df = for_map_gdf[['geo_id', variable, 'percentage']]

geo_json = for_map_gdf.to_crs(epsg="4326").to_json()

for_map_gdf.head(10)

In [ ]:
m = folium.Map(location=[41.2674519, -95.9893462], zoom_start=10)

folium.Choropleth(
    geo_data=for_map_gdf,
    name='Leaving for work between 12:00am and 4:59am',
    data=map_df,
    columns=['geo_id', variable],
    key_on='feature.properties.geo_id',
    fill_color='YlGn',
    fill_opacity=0.6,
    line_opacity=0.2,
    reset=True,
    legend_name="Population Size"
).add_to(m)

m

## Aggegating up to Tracts

In [ ]:
tract_cbsa_gdf = cbsa_gdf.copy(deep=True)

tract_cbsa_gdf['tract_geo_id'] = tract_cbsa_gdf['state'] + tract_cbsa_gdf['county'] + tract_cbsa_gdf['tract']

tract_cbsa_gdf = tract_cbsa_gdf.dissolve(by='tract_geo_id', aggfunc='sum')

tract_cbsa_gdf = tract_cbsa_gdf.reset_index()

tract_cbsa_gdf = tract_cbsa_gdf[tract_cbsa_gdf['tract_geo_id'].str.startswith('31')]

tract_cbsa_gdf.head(5)

In [ ]:
f, ax = plt.subplots(1, figsize=(16, 16))
ax = tract_cbsa_gdf.plot(ax=ax)
plt.show()

In [ ]:
m = folium.Map(location=[41.2674519, -95.9893462], zoom_start=10)

folium.Choropleth(
    geo_data=tract_cbsa_gdf,
    data=tract_cbsa_gdf,
    columns=['tract_geo_id', variable],
    key_on='feature.properties.tract_geo_id',
    fill_color='YlGn',
    fill_opacity=0.6,
    line_opacity=0.2,
    reset=True,
    legend_name="Population Size"
).add_to(m)

m

## Geospatial Analysis

In [ ]:
%matplotlib inline

import seaborn as sns
import pysal as ps
from sklearn import cluster
from sklearn.preprocessing import scale

In [ ]:
filter_columns = lambda x: x.startswith('b') and x.endswith('e')

columns = list(set(df.columns).difference(set(PREFIX_COLUMNS)))

retain_columns = sorted(list(filter(filter_columns, columns)))

df2 = df[PREFIX_COLUMNS + retain_columns].copy(deep=True)

for col in filter(filter_columns, df2.columns):
    df2[col] = df2[col].astype('int')
 
df2['geo_id'] = df2['state'] + df2['county'] + df2['tract'] + df2['block group']

ne_gdf = ne_gdf[ne_gdf['COUNTYFP'].isin(('055', '025', '153', '177'))]
ia_gdf = ia_gdf[ia_gdf['COUNTYFP'].isin(('155',))]

cbsa_gdf = pd.concat([ne_gdf, ia_gdf])

In [ ]:
df2.index = df2['geo_id']

variables = [ 
    'b08302_002e', 
    'b08302_003e', 
    'b08302_004e',
    'b08302_005e', 
    'b08302_006e', 
    'b08302_007e',
    'b08302_008e',
    'b08302_009e',
    'b08302_010e',
    'b08302_011e',
    'b08302_012e',
    'b08302_013e',
    'b08302_014e',
    'b08302_015e'
]

df2 = df2[variables]

df2.head(10)

In [ ]:
db = pd.DataFrame(
    scale(df2),
    index=df2.index,
    columns=df2.columns).rename(lambda x: str(int(x)))

db.head(10)

In [ ]:
geo_db = cbsa_gdf.merge(db, right_index=True, left_on='GEOID')

geo_db = geo_db[['GEOID', 'b08302_002e', 'b08302_003e', 'b08302_004e',
       'b08302_005e', 'b08302_006e', 'b08302_007e', 'b08302_008e',
       'b08302_009e', 'b08302_010e', 'b08302_011e', 'b08302_012e',
       'b08302_013e', 'b08302_014e', 'b08302_015e', 'geometry']]

# # filter to just douglas county
geo_db = geo_db[geo_db['GEOID'].str.startswith('31055')]

geo_db.idex = geo_db['GEOID']

geo_db.head(4)

In [ ]:
cluster.KMeans?

In [ ]:
km5 = cluster.KMeans(n_clusters=5)

km5cls = km5.fit(geo_db.drop(['geometry', 'GEOID'], axis=1).values)

In [ ]:
f, ax = plt.subplots(1, figsize=(15, 15))

geo_db.assign(cl=km5cls.labels_).plot(
    column='cl', 
    categorical=True, 
    legend=True, 
    linewidth=0.1,
    edgecolor='white',
    ax=ax)

ax.set_axis_off()

plt.show()

In [ ]:
geo_db = geo_db.assign(cl=km5cls.labels_)
geo_db.head(10)

In [ ]:
geo_db['cl'].values

In [ ]:
geo_db = geo_db[['GEOID', 'geometry', 'cl']]

m = folium.Map(location=[41.2674519, -95.9893462], zoom_start=10)

bins = [0, 1, 2, 3, 4]

folium.Choropleth(
    geo_data=geo_db,
    name='Leaving for work between 12:00am and 4:59am',
    data=geo_db,
    columns=['GEOID', "cl"],
    key_on='feature.properties.GEOID',
    fill_color='YlGn',
    fill_opacity=0.6,
    line_opacity=0.2,
    reset=True,
    legend_name="Cluster",
    bins=4
).add_to(m)

m

In [ ]:
folium.Choropleth?

## References

* [American Community Survey 5-Year Data (2009-2017)](https://www.census.gov/data/developers/data-sets/acs-5year.html)
* [Census API: Datasets](https://api.census.gov/data.html)